## Sale Price Estimation Model Evaluation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [2]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
housing_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [3]:
housing_reg_df =  housing_df.copy()
housing_reg_df = pd.concat([housing_reg_df,pd.get_dummies(housing_reg_df.centralair, 
                                                            prefix="centralair", drop_first=True)], axis=1)
housing_reg_df = pd.concat([housing_reg_df,pd.get_dummies(housing_reg_df.mszoning, 
                                                            prefix="mszoning", drop_first=True)], axis=1)
housing_reg_df = pd.concat([housing_reg_df,pd.get_dummies(housing_reg_df.bldgtype, 
                                                            prefix="bldgtype", drop_first=True)], axis=1)
housing_reg_df = pd.concat([housing_reg_df,pd.get_dummies(housing_reg_df.exterqual, 
                                                            prefix="exterqual", drop_first=True)], axis=1)
housing_reg_df = pd.concat([housing_reg_df,pd.get_dummies(housing_reg_df.bsmtqual, 
                                                            prefix="bsmtqual", drop_first=True)], axis=1)
housing_reg_df = pd.concat([housing_reg_df,pd.get_dummies(housing_reg_df.salecondition, 
                                                            prefix="salecondition", drop_first=True)], axis=1)

In [4]:
# Y is the target variable
Y = housing_reg_df['saleprice']
# X is the feature set
X = housing_reg_df[['overallqual', 'grlivarea', 'garagecars', 'totalbsmtsf', 'centralair_Y', 'bldgtype_Duplex',
                     'bldgtype_Twnhs','bldgtype_TwnhsE','exterqual_Gd','exterqual_TA','bsmtqual_Fa','bsmtqual_Gd',
                     'bsmtqual_TA','salecondition_Normal','salecondition_Partial']]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.802
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                     391.1
Date:                Sun, 23 Jun 2019   Prob (F-statistic):               0.00
Time:                        20:30:30   Log-Likelihood:                -17360.
No. Observations:                1460   AIC:                         3.475e+04
Df Residuals:                    1444   BIC:                         3.484e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -7141.4517   9585.443     -0.745      0.456   -2.59e+04    1.17e+04
overallqual            1.671e+04   1174.129     14.231      0.000    1.44e+04     1.9e+04
grlivarea                45.0136      2.375     18.949      0.000      40.354      49.673
garagecars             1.345e+04   1675.413      8.025      0.000    1.02e+04    1.67e+04
totalbsmtsf              29.1075      2.695     10.802      0.000      23.822      34.393
centralair_Y           2.132e+04   4067.847      5.241      0.000    1.33e+04    2.93e+04
bldgtype_Duplex       -2.464e+04   5302.808     -4.646      0.000    -3.5e+04   -1.42e+04
bldgtype_Twnhs        -2.148e+04   5633.936     -3.813      0.000   -3.25e+04   -1.04e+04
bldgtype_TwnhsE       -1.668e+04   3696.526     -4.512      0.000   -2.39e+04   -9427.665
exterqual_Gd          -2.356e+04   5225.885     -4.508      0.000   -3.38e+04   -1.33e+04
exterqual_TA          -3.391e+04   5528.691     -6.133      0.000   -4.48e+04   -2.31e+04
bsmtqual_Fa           -4.927e+04   7114.394     -6.926      0.000   -6.32e+04   -3.53e+04
bsmtqual_Gd           -3.279e+04   3605.426     -9.095      0.000   -3.99e+04   -2.57e+04
bsmtqual_TA           -4.124e+04   3986.902    -10.344      0.000   -4.91e+04   -3.34e+04
salecondition_Normal   8698.1326   3250.440      2.676      0.008    2322.044    1.51e+04
salecondition_Partial  1.688e+04   4829.464      3.494      0.000    7401.971    2.63e+04
==============================================================================
Omnibus:                      738.694   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           101640.888
Skew:                          -1.318   Prob(JB):                         0.00
Kurtosis:                      43.791   Cond. No.                     2.31e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.31e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

- R-squared and adjusted R-squared of the model are 0.802 and 0.800 respectively.
- F statistic and its associated p-value are 391.1 and 0.00 respectively.
- AIC and BIC of the model are 34750 and 34840 respectively.

According to the R-squared almost 80% of the variance in the target variable is explained by the model. Hence 20% of the variance in the target is not explained. In this sense, there is a large room for improvement.

In order to increase the goodness of fit of our model:
- We first log transform the dependent variable, applying log(1+x) transformation of NumPy.
- Second, create another variable by summing up the basement, first and the second floor areas.
- Third, add the interaction of the total area and the overall quality of the house.

In [7]:


housing_reg_df['totalsf'] = housing_reg_df['totalbsmtsf'] + housing_reg_df['firstflrsf'] + housing_reg_df['secondflrsf']

housing_reg_df['int_over_sf'] = housing_reg_df['totalsf'] * housing_reg_df['overallqual']

# Y is the target variable
Y = np.log1p(housing_reg_df['saleprice'])
# X is the feature set
X = housing_reg_df[['overallqual', 'grlivarea', 'garagecars','overallqual', 'grlivarea', 'garagecars', 'totalbsmtsf', 'centralair_Y', 'bldgtype_Duplex',
                     'bldgtype_Twnhs','bldgtype_TwnhsE','exterqual_Gd','exterqual_TA','bsmtqual_Fa','bsmtqual_Gd',
                     'bsmtqual_TA','salecondition_Normal','salecondition_Partial', 'totalsf', 'int_over_sf']]
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.847
Model:                            OLS   Adj. R-squared:                  0.845
Method:                 Least Squares   F-statistic:                     469.2
Date:                Sun, 23 Jun 2019   Prob (F-statistic):               0.00
Time:                        20:37:07   Log-Likelihood:                 638.54
No. Observations:                1460   AIC:                            -1241.
Df Residuals:                    1442   BIC:                            -1146.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    10.2575      0.054    191.056      0.000      10.152      10.363
overallqual               0.0854      0.004     19.574      0.000       0.077       0.094
grlivarea             -1.646e-05   4.31e-05     -0.382      0.703      -0.000    6.81e-05
garagecars                0.0387      0.004     10.327      0.000       0.031       0.046
overallqual               0.0854      0.004     19.574      0.000       0.077       0.094
grlivarea             -1.646e-05   4.31e-05     -0.382      0.703      -0.000    6.81e-05
garagecars                0.0387      0.004     10.327      0.000       0.031       0.046
totalbsmtsf              -0.0001   8.94e-05     -1.233      0.218      -0.000    6.52e-05
centralair_Y              0.2071      0.018     11.429      0.000       0.172       0.243
bldgtype_Duplex          -0.1096      0.024     -4.618      0.000      -0.156      -0.063
bldgtype_Twnhs           -0.1510      0.025     -6.045      0.000      -0.200      -0.102
bldgtype_TwnhsE          -0.0832      0.016     -5.075      0.000      -0.115      -0.051
exterqual_Gd             -0.0398      0.024     -1.656      0.098      -0.087       0.007
exterqual_TA             -0.0911      0.025     -3.585      0.000      -0.141      -0.041
bsmtqual_Fa              -0.2477      0.032     -7.756      0.000      -0.310      -0.185
bsmtqual_Gd              -0.1039      0.017     -6.113      0.000      -0.137      -0.071
bsmtqual_TA              -0.1735      0.019     -9.252      0.000      -0.210      -0.137
salecondition_Normal      0.0895      0.014      6.207      0.000       0.061       0.118
salecondition_Partial     0.1114      0.021      5.202      0.000       0.069       0.153
totalsf                   0.0005   9.08e-05      5.255      0.000       0.000       0.001
int_over_sf           -3.265e-05   3.02e-06    -10.795      0.000   -3.86e-05   -2.67e-05
==============================================================================
Omnibus:                      422.695   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3497.733
Skew:                          -1.109   Prob(JB):                         0.00
Kurtosis:                      10.251   Cond. No.                     2.34e+20
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 9.55e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

- R-squared and adjusted R-squared of the model are 0.847 and 0.845 respectively. These are improvements upon the first model.
- F statistic and its associated p-value are 469.2 and 0.00 respectively. This also indicates that the second model is better than the first one.
- AIC and BIC of the model are -1241 and -1146 respectively. These values are lower than the ones of the first model. Hence, the second model is better than the first model.

#### By the lower AIC and BIC, the greater and significant F-statistic, the second estimation model has better goodness of fit than the first model.